In [11]:

import numpy as np
import pandas as pd
import os
print(os.listdir("data/"))

['ArticlesApril2017.csv', 'ArticlesApril2018.csv', 'ArticlesFeb2017.csv', 'ArticlesFeb2018.csv', 'ArticlesJan2017.csv', 'ArticlesJan2018.csv', 'ArticlesMarch2017.csv', 'ArticlesMarch2018.csv', 'ArticlesMay2017.csv', 'CommentsApril2017.csv', 'CommentsApril2018.csv', 'CommentsFeb2017.csv', 'CommentsFeb2018.csv', 'CommentsJan2017.csv', 'CommentsJan2018.csv', 'CommentsMarch2017.csv', 'CommentsMarch2018.csv', 'CommentsMay2017.csv']


Text Generation using LSTMs

Text Generation is a type of Language Modelling problem. Language Modelling is the core problem for a number of of natural language processing tasks such as speech to text, conversational system, and text summarization. A trained language model learns the likelihood of occurrence of a word based on the previous sequence of words used in the text. Language models can be operated at character level, n-gram level, sentence level or even paragraph level. In this notebook, I will explain how to create a language model for generating natural language text by implement and training state-of-the-art Recurrent Neural Network

Generating News headlines

In this kernel, I will be using the dataset of New York Times Comments and Headlines to train a text generation language model which can be used to generate News Headlines

In [12]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras.utils as ku
from keras.callbacks import EarlyStopping

# set seeds for reproducability

from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import string
import os
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore',  category=FutureWarning)

In [13]:
currr_dir = "data/"
all_headlines = []

x=0

for filename in os.listdir(currr_dir):
    if 'Articles' in filename:
        
        if x==0:
            print(filename)
            
        article_df = pd.read_csv(currr_dir+filename)
        
        if x==0:
            print(article_df.shape)
            print(article_df.columns)
            print(article_df.head(5))
            print(article_df.tail(5))
            
        all_headlines.extend(list(article_df.headline.values))
        
        if x==0:
            print(article_df.headline)
            print(article_df.headline.values)
            
        x=1
        
        break

all_headlines = [ h for h in all_headlines if h != 'Unknown' ]
print(len(all_headlines))
print(all_headlines[:5])

ArticlesApril2017.csv
(886, 16)
Index(['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType',
       'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')
  abstract                 articleID  articleWordCount  \
0      NaN  58def1347c459f24986d7c80               716   
1      NaN  58def3237c459f24986d7c84               823   
2      NaN  58def9f57c459f24986d7c90               575   
3      NaN  58defd317c459f24986d7c95              1374   
4      NaN  58df09b77c459f24986d7ca7               708   

                                byline documentType  \
0  By STEPHEN HILTNER and SUSAN LEHMAN      article   
1                      By GAIL COLLINS      article   
2               By THE EDITORIAL BOARD      article   
3                    By MICHAEL POWELL      article   
4                         By DEB AMLEN      article   

                                    

Dataset preparation

# Dataset cleaning

In dataset preparation step, we will first perform text cleaning of the data which includes removal of punctuations and lower casing all the words

In [14]:
def clean_text(txt):
    txt = "".join(w for w in txt if w not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii","ignore")
    return txt

In [15]:
print(clean_text("Questions for: ‘Colleges Discover the Rural St.."))

questions for colleges discover the rural st


In [16]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [17]:
corpus = [clean_text(x) for x in all_headlines]
print(corpus[:10])

['finding an expansive view  of a forgotten people in niger', 'and now  the dreaded trump curse', 'venezuelas descent into dictatorship', 'stain permeates basketball blue blood', 'taking things for granted', 'the caged beast awakens', 'an everunfolding story', 'oreilly thrives as settlements add up', 'mouse infestation', 'divide in gop now threatens trump tax plan']


# Generating Sequence of N-gram Tokens

Language modelling requires a sequence input data, as given a sequence (of words/tokens) the aim is the predict next word/token.

The next step is Tokenization. Tokenization is a process of extracting tokens (terms / words) from a corpus. Python’s library Keras has inbuilt model for tokenization which can be used to obtain the tokens and their index in the corpus. After this step, every text document in the dataset is converted into sequence of tokens

In [45]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    q=0
    # tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    # if q==0:
    #     print(total_words)
    #
    # convert data into sequence of tokens
    input_sequences = []
    for line in corpus:
        
        if q==0:
            print(line)
            
        token_list = tokenizer.texts_to_sequences([line])[0]
        
        if q==0:
            print(token_list)
            print(len(token_list))
            
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            
            if q==0:
                print(n_gram_sequence)
                
            input_sequences.append(n_gram_sequence)
            
            if q==0:
                print(input_sequences)
                
            q+=1

    
    return input_sequences, total_words

In [46]:
input_sequence, total_words = get_sequence_of_tokens(corpus)


finding an expansive view  of a forgotten people in niger
[169, 17, 665, 367, 4, 2, 666, 170, 5, 667]
10
[169, 17]
[[169, 17]]


In [20]:
(input_sequence[:10])

[[169, 17],
 [169, 17, 665],
 [169, 17, 665, 367],
 [169, 17, 665, 367, 4],
 [169, 17, 665, 367, 4, 2],
 [169, 17, 665, 367, 4, 2, 666],
 [169, 17, 665, 367, 4, 2, 666, 170],
 [169, 17, 665, 367, 4, 2, 666, 170, 5],
 [169, 17, 665, 367, 4, 2, 666, 170, 5, 667],
 [6, 80]]

# Padding the Sequences and obtain Variables : Predictors and Target

Now that we have generated a data-set which contains sequence of tokens, it is possible that different sequences have different lengths. Before starting training the model, we need to pad the sequences and make their lengths equal. We can use pad_sequence function of Kears for this purpose. To input this data into a learning model, we need to create predictors and label. We will create N-grams sequence as predictors and the next word of the N-gram as label

In [21]:
def generate_padded_sequences(input_sequence):
    max_sequence_len = max([len(x) for x in input_sequence])
    input_sequences = np.array(pad_sequences(input_sequence, maxlen=max_sequence_len, padding='pre'))
    predictors, labels = input_sequences[:,:-1], input_sequences[:,-1]
    labels = ku.to_categorical(labels, num_classes=total_words)
    
    return predictors, labels, max_sequence_len

In [22]:
predictors, labels, max_seq_len = generate_padded_sequences(input_sequence)

In [23]:
len(predictors)

4806

In [24]:
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [29]:
(max_seq_len)

19

In [30]:
print(total_words)

2422


# Constructiing model
Now we can obtain the input vector X and the label vector Y which can be used for the training purposes. Recent experiments have shown that recurrent neural networks have shown a good performance in sequence to sequence learning and text data applications

The memory state in RNNs gives an advantage over traditional neural networks but a problem called Vanishing Gradient is associated with them. In this problem, while learning with a large number of layers, it becomes really hard for the network to learn and tune the parameters of the earlier layers. To address this problem, A new type of RNNs called LSTMs (Long Short Term Memory) Models have been developed.

LSTMs have an additional state called ‘cell state’ through which the network makes adjustments in the information flow. The advantage of this state is that the model can remember or forget the leanings more selectively.

Input Layer : Takes the sequence of words as input

LSTM Layer : Computes the output using LSTM units. I have added 100 units in the layer, but this number can be fine tuned later.

Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the LSTM layer. It helps in preventing over fitting. (Optional Layer)

Output Layer : Computes the probability of the best possible next word as output

In [75]:
def create_model(max_seq_len, total_words):
    input_len = max_seq_len - 1
    
    model = Sequential()
    
    # input: embedding layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # hidden: lstm layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # output layer
    model.add(Dense(total_words, activation='softmax'))
    
    # compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

In [76]:
model = create_model(max_seq_len, total_words)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 18, 10)            24220     
                                                                 
 lstm_1 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 2422)              244622    
                                                                 
Total params: 313,242
Trainable params: 313,242
Non-trainable params: 0
_________________________________________________________________


In [77]:
# fitting (training) the model by passing predictors and labels as training data

model.fit(predictors, labels, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

Generating the text

Great, our model architecture is now ready. 

Next lets write the function to predict the next word based on the input words (or seed text). We will first tokenize the seed text, pad the sequences and pass into the trained model to get predicted word. The multiple predicted words can be appended together to get predicted sequence.

In [72]:
def generate_text(seed_text, next_words, model, max_seq_len):
    w=0
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        
        if w==0:
            print(token_list)
            
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        
        if w==0:
            print(token_list)
        
        predict_x=model.predict(token_list, verbose=0)
        predicted=np.argmax(predict_x,axis=1)
        
        if w==0:
            print(predicted)

        output_word = ''

        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        seed_text = seed_text + " " + output_word

        w=1
        
    return seed_text.title()

In [73]:
print(generate_text("united states", 5, model, max_seq_len))

[318, 1157]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0  318 1157]]
[25]
United States This Really Like Trump Stay


In [74]:
print (generate_text("united states", 5, model, max_seq_len))
print (generate_text("preident trump", 4, model, max_seq_len))
print (generate_text("donald trump", 4, model, max_seq_len))
print (generate_text("india and china", 4, model, max_seq_len))
print (generate_text("new york", 4, model, max_seq_len))
print (generate_text("science and technology", 5, model, max_seq_len))

[318, 1157]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0  318 1157]]
[25]
United States This Really Like Trump Stay
[10]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10]]
[9]
Preident Trump Is Wimping If Pay
[282, 10]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 282  10]]
[297]
Donald Trump Fuels Badenov Have Independent
[1935, 6, 185]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0 1935    6  185]]
[2002]
India And China Leftwing Politician Shakes Up
[11, 30]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 11 30]]
[31]
New York Today A Holocaust Survivors
[6, 1840]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    6 1840]]
[2106]
Science And Technology Protested The Vietnam War A


In [25]:
li = [[1,2,3],[4,5,6]]

In [26]:
li

[[1, 2, 3], [4, 5, 6]]

TypeError: list indices must be integers or slices, not tuple